In [1]:
from google.colab import drive

drive.mount('drive', force_remount=True)

Mounted at drive


In [2]:
%cd drive/MyDrive/Experiments/SNLP2/week_08/

/content/drive/MyDrive/Experiments/SNLP2/week_08


In [3]:
!ls

config.yaml			      evaluate_TER.ipynb
convert_data_to_opennmt_format.ipynb  models
data				      openNMT-tutorial.ipynb
data-opennmt			      requirements.txt


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.8/216.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 906.5/906.5 MB 217.7 MB/s eta 0:00:01

### Build Vocab

In [5]:
# !onmt_build_vocab -config config.yaml -n_sample -1

### Train Model

- First save config

In [19]:
!onmt_train -config config.yaml -save_config models/2-bi-lstm-bigger-config.yaml

/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_activations.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_activations.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_losses.py:12: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_losses.py:36: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/onmt/models/sru.py:396: FutureWarning: `to

In [20]:
!onmt_train -config config.yaml

/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_activations.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_activations.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_losses.py:12: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/onmt/modules/sparse_losses.py:36: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/onmt/models/sru.py:396: FutureWarning: `to

### Make Predictions

* GPU usage is recommended

In [21]:
!onmt_translate -model models/2-bi-lstm-bigger_step_3300.pt -src data-opennmt/test/slk.gold.src -output data-opennmt/predictions/2-bi-lstm-bigger_step_3300.predictions -gpu 0 -verbose

Streaming output truncated to the last 5000 lines.
SENT 1001: ['d', 'o', 'f', 'u', 'k', 'o', 'v', 'á', 'v', 'a', 't', 'e', 'ľ', 'n', 'ý', '<ADJ>', '<GEN>', '<CMPR>', '<FEM>', '<PL>']
PRED 1001: d o f u k o v á v a t e ľ n e j š í c h
PRED SCORE: -0.0020

[2025-06-26 12:52:28,937 INFO] 
SENT 1002: ['z', 'b', 'i', 't', 'ý', '<ADJ>', '<ANIM>', '<NOM>', '<SPRL>', '<MASC>', '<PL>']
PRED 1002: n a j z b i t e j š í
PRED SCORE: -0.0069

[2025-06-26 12:52:28,937 INFO] 
SENT 1003: ['z', 'a', 'ú', 't', 'o', 'č', 'i', 'ť', '<V>', '<PFV>', '<NEUT>', '<IND>', '<PL>', '<3>', '<PST>']
PRED 1003: z a ú t o č i l i
PRED SCORE: -0.0017

[2025-06-26 12:52:28,937 INFO] 
SENT 1004: ['z', 'j', 'a', 'v', 'o', 'v', 'á', 'v', 'a', 'ť', '<V>', '<INAN>', '<PFV>', '<MASC>', '<COND>', '<PL>', '<2>', '<PST>']
PRED 1004: b o l i b y s t e z j a v o v á v a l i
PRED SCORE: -0.0329

[2025-06-26 12:52:28,937 INFO] 
SENT 1005: ['d', 'o', 'a', 'l', 'o', 'k', 'o', 'v', 'a', 't', 'e', 'ľ', 'n', 'ý', '<ADJ>', '<ESS>', '<SPR

### Evaluate Model (on TER)

In [24]:
import evaluate
import os

modelnames = os.listdir('data-opennmt/predictions/')

best_ter, best_modelname = 100.0, None
for modelname in modelnames:
    PATH2PREDS = f'data-opennmt/predictions/{modelname}'
    PATH2GOLD = 'data-opennmt/test/slk.gold.tgt'

    metric = evaluate.load('ter')


    predictions, references = [], []

    with open(PATH2PREDS, 'rt', encoding='utf8') as fr1, open(PATH2GOLD, 'rt', encoding='utf8') as fr2:
        preds = fr1.read().split('\n')
        refs = fr2.read().split('\n')
        for p, r in zip(preds, refs):
            p_str, r_str = ''.join(p.split(' ')), ''.join(r.split(' '))
            predictions.append(p_str)
            references.append([r_str])

    ter = metric.compute(predictions=predictions, references=references)

    if best_ter > ter["score"]:
       best_ter = ter["score"]
       best_modelname = modelname

    # TER <= 20 -> high quality translation
    print(f'name: {modelname}\nTER: {ter["score"]}\n'
          f'num.edits: {ter["num_edits"]}\n'
          f'ref.length: {ter["ref_length"]}\n\n')

name: model1_step_110_pred.txt
TER: 100.0
num.edits: 2000
ref.length: 2000.0


name: 3-bi-lstm_step_3300.predictions
TER: 38.4
num.edits: 768
ref.length: 2000.0


name: 2-bi-lstm_step_3300.predictions
TER: 15.5
num.edits: 310
ref.length: 2000.0


name: 2-bi-lstm-smaller_step_3300.predictions
TER: 16.35
num.edits: 327
ref.length: 2000.0


name: transformer-v1_step_3300.predictions
TER: 95.3
num.edits: 1906
ref.length: 2000.0


name: transformer-v1_step_10000.predictions
TER: 82.1
num.edits: 1642
ref.length: 2000.0


name: transformer-v2_step_10000.predictions
TER: 71.45
num.edits: 1429
ref.length: 2000.0


name: transformer-v3_step_10000.predictions
TER: 99.95
num.edits: 1999
ref.length: 2000.0


name: 1-bi-lstm_step_3300.predictions
TER: 25.75
num.edits: 515
ref.length: 2000.0


name: 1-uni-lstm_step_3300.predictions
TER: 31.6
num.edits: 632
ref.length: 2000.0


name: 2-uni-lstm_step_3300.predictions
TER: 24.7
num.edits: 494
ref.length: 2000.0


name: 2-bi-lstm-bigger_step_3300.predict

### Best Model

In [25]:
print(f'name: {best_modelname}, TER: {best_ter}')

name: 2-bi-lstm_step_3300.predictions, TER: 15.5
